# NHGF application demo: Calculating zonal stats for NLDAS zarr data using gdptools

### Overview

This demo calculates an area-weighted mean NLDAS Soil Moisture (kg/m^2) for 10-digit Hydrologic Unit (HU10) region polygons covering the Saint Croix River watershed, which includes the HU4 of 0703 in Minnesota and Wisconsin for one time step within the full time series of gridded hourly NLDAS Mosaic model output. The polygon area-weighted mean calculation is carried out using the [gdptools library](https://code.usgs.gov/wma/nhgf/toolsteam/gdptools), and the results are visualized using hvplot.

### Dependencies

Requirements include fsspec, holoviews, numpy, pandas, geopandas, ujson, s3fs, fsspec, and kerchunk. It is recommended to build a conda environment from the environment.yml file in gdptools in order to be able to use the gdptools for calculating and plotting polygon-weighted mean NLDAS soil moisture values: https://code.usgs.gov/wma/nhgf/toolsteam/gdptools.

```conda env create -f environment.yml```

Once the conda environment is created, activate gdptools, `conda activate gdptools`, and run `poetry install` to build the environment.

For those not using the gdptools environment file, it is recommended to use pip to install kerchunk, as the current version on conda-forge is not up-to-date.

```pip install git+https://github.com/fsspec/kerchunk.git```

Users will also need the [AWS command line interface (CLI) tools](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html), an AWS account, and an [AWS credentials file](https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-files.html) with a profile which has the AWS Access Key ID and AWS Secret Access Key for the NHGF S3 bucket. This notebook assumes there is a profile in the AWS credentials file for the NHGF bucket which has the name "NHGF". This credentials file should be under ~/.aws/.

In [ ]:
import fsspec
import os
import xarray as xr
import holoviews as hv
import hvplot
import hvplot.pandas
import hvplot.xarray
import os
import geopandas as gpd
import pandas as pd
import numpy as np
import cartopy
import pyproj
import time
from datetime import datetime

from gdptools.helpers import generate_weights
from gdptools.helpers import _get_cells_poly
from gdptools.helpers import get_crs
from gdptools.helpers import run_weights

fs = fsspec.filesystem('file') # Set local file system

proj_dir = 's3://nhgf-development/nldas' # NHGF S3 bucket location
json_dir = f'{proj_dir}/data/jsons'

var = 'SOILM'
timedimension = 'time'

# Import Saint Croix Watershed HU10 polygons
hucpoly = '~/data/Saint_Croix_HU10_5070.geojson'
with fs.open(hucpoly, mode='r') as file:
    gdf = gpd.read_file(file)
# gdf = gpd.read_file('../tests/data/hru_1210.shp')
gdf
polygon_index = 'huc10'
print(gdf.crs)

# # Import State Boundary Polygons
statespoly = '~/data/tl_2012_us_state.geojson'
with fs.open(statespoly, mode='r') as file:
    states = gpd.read_file(file)

states.to_crs(epsg='5070', inplace=True)
print(states.crs)

timeidxstart = 1600
timeidxend = timeidxstart + 1

mosaicmultizarr = f'{json_dir}/mosaic_SOILM.json' # Path to zarr virtual dataset for mosaic model soil moisture data

# Storage and read options to access S3 bucket
s_opts = {'requester_pays':True, 'skip_instance_cache':True, 'profile':'NHGF'}
r_opts = {'anon':False, 'profile':'NHGF'}
# Set file system with configuration to access S3 bucket
mosaicfs = fsspec.filesystem(
    'reference', 
    fo=mosaicmultizarr, 
    ref_storage_args=s_opts, 
    remote_protocol='s3', 
    remote_options=r_opts, 
    profile='NHGF')

mosaicmapper = mosaicfs.get_mapper("")
# Open virtual dataset using xarray. This will only load in metadata
mosaic_combo = xr.open_dataset(mosaicmapper, engine="zarr", backend_kwargs={'consolidated':False}, chunks={})
# Extract surface (0-10 meters) soil moisture from dataset
surface_mosaic = mosaic_combo.SOILM[:,0,:,:]
surface_mosaic

In [ ]:
from shapely.geometry import box
bbox = box(*gdf.to_crs(4326).total_bounds)
gdf_bounds = bbox.buffer(2*max(max(np.diff(surface_mosaic.lat.values)), max(np.diff(surface_mosaic.lon.values)))).bounds
gdf_bounds

In [ ]:
import datetime
subset_dict = {'lon': slice(gdf_bounds[0], gdf_bounds[2]),
               'lat': slice(gdf_bounds[1], gdf_bounds[3]),
               'time': slice(pd.to_datetime('1979-01-02T01:00:00.000000000'),pd.to_datetime('1979-01-02T02:00:00.000000000')),
               'depth': 5}
print(subset_dict)
ds_ss = mosaic_combo.sel(**subset_dict)
ds_ss.SOILM.isel(time=0).plot()

In [ ]:
# # Quickly plot selected time slice
# mosaic_combo.SOILM.isel(time=timeidxstart, depth=0).plot()

In [ ]:
# Create separate dataset and select time slice
data = ds_ss
# # Extract date for time slice
# date = str(pd.to_datetime(data.coords['time'].values[0]).date())

In [ ]:
# Vectorize grid cells to polygons 
gridpoly = _get_cells_poly(ds_ss.isel(time=0), x='lon', y='lat', var=var, crs_in=4326)

In [ ]:
gridpoly

In [ ]:
gdf

In [ ]:
# data

In [ ]:
# Create geodataframe from vectorized grid cells
gdf_grid = gridpoly
gdf_grid.to_crs(epsg='5070', inplace=True)

In [ ]:
# Generate weights of the proportion of the grid cells covering each HU10 polygon
weightspath = f'tmp_stcroix_wgts.csv'

wght_n = generate_weights(
    poly=gdf,
    poly_idx=polygon_index,
    grid_cells=gdf_grid,
    grid_cells_crs=5070,
    # grid_cells_crs=4326,
    filename=weightspath,
    wght_gen_crs=5070,
)

In [ ]:
# Calculate weighted soil moisture for HU10 polygons
if isinstance(wght_n, pd.DataFrame):
    newgdf, vals = run_weights(
        var=var,
        time=timedimension,
        ds=data,
        wght_file=wght_n,
        shp=gdf,
        geom_id=polygon_index,
    )

    print(vals)

newgdf[var] = vals[0,:]

In [ ]:
gdf

In [ ]:
# identify area of interest based on the extent of the Saint Croix watershed HU10 subset
bounds = newgdf.total_bounds
print(bounds)

# extract linear unit y and x values
x_min, y_min = bounds[1], bounds[0]
x_max, y_max = bounds[3], bounds[2]

# Convert linear x and y values to longitudes and latitudes
lat_min, lon_min = pyproj.transform(pyproj.Proj('epsg:5070'), pyproj.Proj('epsg:4326'), y_min, x_min)
lat_max, lon_max = pyproj.transform(pyproj.Proj('epsg:5070'), pyproj.Proj('epsg:4326'), y_max, x_max)
print('lon_min', lon_min, 'lon_max', lon_max, 'lat_min', lat_min, 'lat_max', lat_max)

In [ ]:
data.SOILM

In [ ]:
title = f'Saint Croix NLDAS Surface Soil Moisture kg/m^2 for'
rawdata = data.SOILM[0,:,:].hvplot.quadmesh( # Input surface soil moisture grid
    'lon', # x coordinates
    'lat', # y coordinates
    var, # data variable to be displayed, set in first cell
    #crs = 
    projection=cartopy.crs.AlbersEqualArea(-91, 45), # Albers Equal Area centered on Saint Croix watershed
    global_extent=False,  
    ylim=(lat_min-1, lat_max+1), # use slightly larger bounding box than that retrieved from the gdf dataset
    xlim=(lon_min-1, lon_max+1), 
    clim=(12,30), # dataset visualization graduation range for soil moisture
    alpha=0.5)
poly_weighted = newgdf.hvplot.polygons( # HU10 watershed catchment polygon-weighted surface soil moisture
    c=var, # data variable to be displayed, set in first cell
    crs=5070, # Albers Equal Area
    line_width=0.5, 
    line_color='white', 
    clim=(12,30), # dataset visualization graduation range for soil moisture
    alpha=1, 
    label = "Saint Croix HU10") # add toggleable legend element, adding the option to make the polygons mostly transparent in order to compare against the gridded data
# stateshv = states.hvplot.polygons( # state outline dataset
#     crs=5070, 
#     line_width=0.5, 
#     line_color='black', 
#     alpha=1, 
#     color=None)

In [ ]:
# Combine layers into a single map
combinedmap = (poly_weighted * rawdata).relabel(title).opts(legend_position='bottom', width=500, height=500)
combinedmap